# Explore Python PEP data using littletable

Load and examine a zipped JSON extract of data on Python PEPs.
 - access data by unique `id` key
 - access table using normal slice notation
 - present table in tabular, JSON, or CSV output
 - create a full-text search index on the abstract field

In [3]:
import littletable as lt

In [23]:
# examine first 3 rows of data (except for "abstract" field)
peek = lt.Table().json_import("peps.json.zip")
peek.info()

{'len': 642,
 'name': 'peps.json.zip',
 'fields': ['id',
  'title',
  'authors',
  'discussions_to',
  'status',
  'type',
  'topic',
  'created',
  'python_version',
  'post_history',
  'resolution',
  'requires',
  'replaces',
  'superseded_by',
  'url',
  'abstract'],
 'indexes': [],
 'created': datetime.datetime(2024, 3, 25, 20, 6, 51, 891354, tzinfo=datetime.timezone.utc),
 'modified': datetime.datetime(2024, 3, 25, 20, 6, 51, 898138, tzinfo=datetime.timezone.utc),
 'last_import': datetime.datetime(2024, 3, 25, 20, 6, 51, 898138, tzinfo=datetime.timezone.utc)}

In [25]:
# look at the first 3 rows, omitting the "abstract" field
peek.select("-abstract")[:3].present()

                                                   peps.json.zip                                                   
+-----------------------------------------------------------------------------------------------------------------+
|    |       |       | Disc… |       |       |       |       | Pyth… | Post  |      |       |      | Supe… |      |
| Id | Title | Auth… |  To   | Stat… | Type  | Topic | Crea… | Vers… | Hist… | Res… | Requ… | Rep… |  By   | Url  |
|----+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+------+-------+------|
|  0 | Index | pyth… |       | Acti… | Info… |       | 13-J… |       |       |      |       |      |       | htt… |
|    | of    |       |       |       |       |       |       |       |       |      |       |      |       |      |
|    | Pyth… |       |       |       |       |       |       |       |       |      |       |      |       |      |
|    | Enha… |       |       |       |       |       |       |       |       |      |       |      |       |      |
|    | Prop… |       |       |       |       |       |       |       |       |      |       |      |       |      |
|    | (PEP… |       |       |       |       |       |       |       |       |      |       |      |       |      |
|  1 | PEP   | Wars… |       | Acti… | Proc… |       | 13-J… |       | 21-M… |      |       |      |       | htt… |
|    | Purp… | Hylt… |       |       |       |       |       |       | 29-J… |      |       |      |       |      |
|    | and   | Good… |       |       |       |       |       |       | 03-M… |      |       |      |       |      |
|    | Guid… | Cogh… |       |       |       |       |       |       | 05-M… |      |       |      |       |      |
|    |       |       |       |       |       |       |       |       | 07-A… |      |       |      |       |      |
|  2 | Proc… | Cann… |       | Acti… | Proc… |       | 07-J… |       | 07-J… |      |       |      |       | htt… |
|    | for   | Faas… |       |       |       |       |       |       | 09-M… |      |       |      |       |      |
|    | Addi… |       |       |       |       |       |       |       |       |      |       |      |       |      |
|    | New   |       |       |       |       |       |       |       |       |      |       |      |       |      |
|    | Modu… |       |       |       |       |       |       |       |       |      |       |      |       |      |
+-----------------------------------------------------------------------------------------------------------------+

In [24]:
# import PEP data from zipped JSON, converting id's to ints and created date stings to Python datetimes
peps = lt.Table().json_import(
    "peps.json.zip",
    transforms={
        "id": int,
        "created": lt.Table.parse_date("%d-%b-%Y"),
    }
)

In [7]:
# display summary Table info
peps.info()

{'len': 642,
 'name': 'peps.json.zip',
 'fields': ['id',
  'title',
  'authors',
  'discussions_to',
  'status',
  'type',
  'topic',
  'created',
  'python_version',
  'post_history',
  'resolution',
  'requires',
  'replaces',
  'superseded_by',
  'url',
  'abstract'],
 'indexes': [],
 'created': datetime.datetime(2024, 3, 25, 19, 59, 19, 196974, tzinfo=datetime.timezone.utc),
 'modified': datetime.datetime(2024, 3, 25, 19, 59, 19, 210519, tzinfo=datetime.timezone.utc),
 'last_import': datetime.datetime(2024, 3, 25, 19, 59, 19, 210519, tzinfo=datetime.timezone.utc)}

In [8]:
# access records by unique PEP id
peps.create_index("id", unique=True)
print("PEP20:", peps.by.id[20].title)

PEP20: The Zen of Python


In [9]:
# add a numeric "year" field, and index it (non-unique index, since there are multiple PEPs in any given year)
peps.add_field("year", lambda pep: pep.created.year)
peps.create_index("year")

In [11]:
# present PEPs created in 2016 - access index by indexed field name
peps.by.year[2016]("PEPs Created in 2016").select("id python_version title status url").present()

                                               PEPs Created in 2016                                                
+-----------------------------------------------------------------------------------------------------------------+
|  Id | Python Version | Title                                   | Status     | Url                               |
|-----+----------------+-----------------------------------------+------------+-----------------------------------|
| 509 | 3.6            | Add a private version to dict           | Final      | https://peps.python.org/pep-0509/ |
| 510 | 3.6            | Specialize functions with guards        | Rejected   | https://peps.python.org/pep-0510/ |
| 511 | 3.6            | API for code transformers               | Rejected   | https://peps.python.org/pep-0511/ |
| 513 |                | A Platform Tag for Portable Linux Built | Superseded | https://peps.python.org/pep-0513/ |
|     |                | Distributions                           |            |                                   |
| 514 |                | Python registration in the Windows      | Active     | https://peps.python.org/pep-0514/ |
|     |                | registry                                |            |                                   |
| 515 | 3.6            | Underscores in Numeric Literals         | Final      | https://peps.python.org/pep-0515/ |
| 518 |                | Specifying Minimum Build System         | Final      | https://peps.python.org/pep-0518/ |
|     |                | Requirements for Python Projects        |            |                                   |
| 519 | 3.6            | Adding a file system path protocol      | Final      | https://peps.python.org/pep-0519/ |
| 520 | 3.6            | Preserving Class Attribute Definition   | Final      | https://peps.python.org/pep-0520/ |
|     |                | Order                                   |            |                                   |
| 522 | 3.6            | Allow BlockingIOError in security       | Rejected   | https://peps.python.org/pep-0522/ |
|     |                | sensitive APIs                          |            |                                   |
| 523 | 3.6            | Adding a frame evaluation API to        | Final      | https://peps.python.org/pep-0523/ |
|     |                | CPython                                 |            |                                   |
| 524 | 3.6            | Make os.urandom() blocking on Linux     | Final      | https://peps.python.org/pep-0524/ |
| 525 | 3.6            | Asynchronous Generators                 | Final      | https://peps.python.org/pep-0525/ |
| 526 | 3.6            | Syntax for Variable Annotations         | Final      | https://peps.python.org/pep-0526/ |
| 527 |                | Removing Un(der)used file               | Final      | https://peps.python.org/pep-0527/ |
|     |                | types/extensions on PyPI                |            |                                   |
| 528 | 3.6            | Change Windows console encoding to      | Final      | https://peps.python.org/pep-0528/ |
|     |                | UTF-8                                   |            |                                   |
| 529 | 3.6            | Change Windows filesystem encoding to   | Final      | https://peps.python.org/pep-0529/ |
|     |                | UTF-8                                   |            |                                   |
| 530 | 3.6            | Asynchronous Comprehensions             | Final      | https://peps.python.org/pep-0530/ |
| 531 | 3.7            | Existence checking operators            | Withdrawn  | https://peps.python.org/pep-0531/ |
| 532 | 3.8            | A circuit breaking protocol and binary  | Deferred   | https://peps.python.org/pep-0532/ |
|     |                | operators                               |            |                                   |
| 533 |                | Deterministic cleanup for itera

In [12]:
# how many PEPs since 2020? use slice notation for ">="
print(f"Number of PEPs since 2020: {len(peps.by.year[2020:])}")

Number of PEPs since 2020: 130


In [13]:
# pivot by year and dump counts, or present as nice table
peps.pivot("year").dump_counts()

Pivot: year
2000         41
2001         52
2002         30
2003         27
2004         11
2005         16
2006         23
2007         32
2008         10
2009         25
2010          7
2011         16
2012         28
2013         27
2014         24
2015         29
2016         30
2017         26
2018         26
2019         32
2020         36
2021         30
2022         25
2023         35
2024          4


In [14]:
peps_by_year = peps.pivot("year").as_table()("PEPs by Year Submitted")
peps_by_year.add_field("", lambda rec: "o" * rec.count)
peps_by_year.present(caption="as of 21 March 2024", caption_justify="left")

                        PEPs by Year Submitted                         
+---------------------------------------------------------------------+
| Year | Count |                                                      |
|------+-------+------------------------------------------------------|
| 2000 |    41 | ooooooooooooooooooooooooooooooooooooooooo            |
| 2001 |    52 | oooooooooooooooooooooooooooooooooooooooooooooooooooo |
| 2002 |    30 | oooooooooooooooooooooooooooooo                       |
| 2003 |    27 | ooooooooooooooooooooooooooo                          |
| 2004 |    11 | ooooooooooo                                          |
| 2005 |    16 | oooooooooooooooo                                     |
| 2006 |    23 | ooooooooooooooooooooooo                              |
| 2007 |    32 | oooooooooooooooooooooooooooooooo                     |
| 2008 |    10 | oooooooooo                                           |
| 2009 |    25 | ooooooooooooooooooooooooo                            |
| 2010 |     7 | ooooooo                                              |
| 2011 |    16 | oooooooooooooooo                                     |
| 2012 |    28 | oooooooooooooooooooooooooooo                         |
| 2013 |    27 | ooooooooooooooooooooooooooo                          |
| 2014 |    24 | oooooooooooooooooooooooo                             |
| 2015 |    29 | ooooooooooooooooooooooooooooo                        |
| 2016 |    30 | oooooooooooooooooooooooooooooo                       |
| 2017 |    26 | oooooooooooooooooooooooooo                           |
| 2018 |    26 | oooooooooooooooooooooooooo                           |
| 2019 |    32 | oooooooooooooooooooooooooooooooo                     |
| 2020 |    36 | oooooooooooooooooooooooooooooooooooo                 |
| 2021 |    30 | oooooooooooooooooooooooooooooo                       |
| 2022 |    25 | ooooooooooooooooooooooooo                            |
| 2023 |    35 | ooooooooooooooooooooooooooooooooooo                  |
| 2024 |     4 | oooo                                                 |
+---------------------------------------------------------------------+
as of 21 March 2024                                                    

In [15]:
# create full text search on PEP abstracts
peps.create_search_index("abstract")

In [21]:
# search by search field name
walrus_pep = peps.search.abstract("walrus", as_table=True)("'walrus' Search Results")
walrus_pep.select("id title year authors").present()

                    'walrus' Search Results                    
+-------------------------------------------------------------+
|  Id | Title                  | Year | Authors               |
|-----+------------------------+------+-----------------------|
| 572 | Assignment Expressions | 2018 | Angelico, Peters, GvR |
+-------------------------------------------------------------+

In [18]:
# display as JSON (json_export also accepts an output file)
print(walrus_pep.select("id title year authors").json_export())

[
{"id": 572, "title": "Assignment Expressions", "year": 2018, "authors": "Angelico, Peters, GvR"}
]


In [19]:
# search for PEPs referring to GvR or Guido or BDFL
bdfl_peps = peps.search.abstract("gvr guido bdfl", as_table=True)("GvR PEPs")
bdfl_peps.orderby("id")
bdfl_peps.select("id title year url authors").present()

                                                     GvR PEPs                                                      
+-----------------------------------------------------------------------------------------------------------------+
|   Id | Title                          | Year | Url                             | Authors                        |
|------+--------------------------------+------+---------------------------------+--------------------------------|
|    8 | Style Guide for Python Code    | 2001 | https://peps.python.org/pep-00… | GvR, Warsaw, Coghlan           |
|   20 | The Zen of Python              | 2004 | https://peps.python.org/pep-00… | Peters                         |
|   42 | Feature Requests               | 2000 | https://peps.python.org/pep-00… | Hylton                         |
|  102 | Doing Python Micro Releases    | 2002 | https://peps.python.org/pep-01… | Baxter, Warsaw, GvR            |
|  230 | Warning Framework              | 2000 | https://peps.python.org/pep-02… | GvR                            |
|  246 | Object Adaptation              | 2001 | https://peps.python.org/pep-02… | Martelli, Evans                |
|  298 | The Locked Buffer Interface    | 2002 | https://peps.python.org/pep-02… | Heller                         |
|  308 | Conditional Expressions        | 2003 | https://peps.python.org/pep-03… | GvR, Hettinger                 |
|  346 | User Defined ("``with``")      | 2005 | https://peps.python.org/pep-03… | Coghlan                        |
|      | Statements                     |      |                                 |                                |
|  366 | Main module explicit relative  | 2007 | https://peps.python.org/pep-03… | Coghlan                        |
|      | imports                        |      |                                 |                                |
|  401 | BDFL Retirement                | 2009 | https://peps.python.org/pep-04… | Warsaw, Cannon                 |
|  568 | Generator-sensitivity for      | 2018 | https://peps.python.org/pep-05… | Smith                          |
|      | Context Variables              |      |                                 |                                |
| 3099 | Things that will Not Change in | 2006 | https://peps.python.org/pep-30… | Brandl                         |
|      | Python 3000                    |      |                                 |                                |
| 3100 | Miscellaneous Python 3.0 Plans | 2004 | https://peps.python.org/pep-31… | Cannon                         |
| 3111 | Simple input built-in in       | 2006 | https://peps.python.org/pep-31… | Roberge                        |
|      | Python 3000                    |      |                                 |                                |
| 3146 | Merging Unladen Swallow into   | 2010 | https://peps.python.org/pep-31… | Winter, Yasskin, Kleckner      |
|      | CPython                        |      |                                 |                                |
| 8000 | Python Language Governance     | 2018 | https://peps.python.org/pep-80… | Warsaw                         |
|      | Proposal Overview              |      |                                 |                                |
| 8001 | Python Governance Voting       | 2018 | https://peps.python.org/pep-80… | Cannon, Heimes, Stufft, Snow,  |
|      | Process                        |      |                                 | Smith, Langa, Mariatta, Smith, |
|      |                                |      |                                 | Salgado, Hettinger, Einat,     |
|      |                                |      |                                 | Peters, Ware                   |
| 8002 | Open Source Governance Survey  | 2018 | https://peps.python.org/pep-80… | Warsaw, Langa, Pitrou,         |
|      |                                |      |                                 | Hellmann, Willing              |
| 8010 | The Technical Leader           | 2018 | https:/

In [26]:
# define a custom JSON encoder for datetime.date field
import json
class JsonDateEncoder(json.JSONEncoder):
    def default(self, o):
        import datetime
        if isinstance(o, datetime.date):
            return str(o)
        return super().default(o)

print(bdfl_peps.select("id title created").json_export(json_encoder=(JsonDateEncoder,)))

[
{"id": 8, "title": "Style Guide for Python Code", "created": "2001-07-05"},
{"id": 20, "title": "The Zen of Python", "created": "2004-08-19"},
{"id": 42, "title": "Feature Requests", "created": "2000-09-12"},
{"id": 102, "title": "Doing Python Micro Releases", "created": "2002-01-09"},
{"id": 230, "title": "Warning Framework", "created": "2000-11-28"},
{"id": 246, "title": "Object Adaptation", "created": "2001-03-21"},
{"id": 298, "title": "The Locked Buffer Interface", "created": "2002-07-26"},
{"id": 308, "title": "Conditional Expressions", "created": "2003-02-07"},
{"id": 346, "title": "User Defined (\"``with``\") Statements", "created": "2005-05-06"},
{"id": 366, "title": "Main module explicit relative imports", "created": "2007-05-01"},
{"id": 401, "title": "BDFL Retirement", "created": "2009-04-01"},
{"id": 568, "title": "Generator-sensitivity for Context Variables", "created": "2018-01-04"},
{"id": 3099, "title": "Things that will Not Change in Python 3000", "created": "2006-0

In [29]:
# CSV import/export also supported
print(bdfl_peps.select("id title created").csv_export())

id,title,created
8,Style Guide for Python Code,2001-07-05
20,The Zen of Python,2004-08-19
42,Feature Requests,2000-09-12
102,Doing Python Micro Releases,2002-01-09
230,Warning Framework,2000-11-28
246,Object Adaptation,2001-03-21
298,The Locked Buffer Interface,2002-07-26
308,Conditional Expressions,2003-02-07
346,"User Defined (""``with``"") Statements",2005-05-06
366,Main module explicit relative imports,2007-05-01
401,BDFL Retirement,2009-04-01
568,Generator-sensitivity for Context Variables,2018-01-04
3099,Things that will Not Change in Python 3000,2006-04-04
3100,Miscellaneous Python 3.0 Plans,2004-08-20
3111,Simple input built-in in Python 3000,2006-09-13
3146,Merging Unladen Swallow into CPython,2010-01-01
8000,Python Language Governance Proposal Overview,2018-08-24
8001,Python Governance Voting Process,2018-08-24
8002,Open Source Governance Survey,2018-08-24
8010,The Technical Leader Governance Model,2018-08-24
8014,The Commons Governance Model,2018-09-16
8015,Organization of t